In [1]:
import numpy as np
import psycopg2
from psycopg2 import Error
from tabulate import tabulate
import numpy as np

connection = psycopg2.connect(user="westwjbipxacva",
                                  password="1d73874df84b39cab4f5d5c8f44514b0ef2244cd7bd7bc0596f323f0da2c1924",
                                  host="ec2-54-146-82-179.compute-1.amazonaws.com",
                                  port="5432",
                                  database="d5gtucomvgir2l")

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def cat_utility(ds, clustering, m):
  # category utility of clustering of dataset ds
  n = len(ds)  # number items
  d = len(ds[0])  # number attributes/dimensions

  # get number items in each cluster
  cluster_cts = [0] * m  # [0,0]
  for ni in range(n):  # each item
    k = clustering[ni]
    cluster_cts[k] += 1

  for i in range(m): 
    if cluster_cts[i] == 0:   # a cluster has no items
      return 0.0

  # get number unique values, each att
  # ex: [3, 3, 2] -> 3 colors, 3 lengths, 2 weights
  # same as max+1 in ds if ds is encoded
  # used only for list allocation
  unique_vals = [0] * d  # [0,0,0]
  for i in range(d):  # each att/dim
    maxi = 0
    for ni in range(n):  # each item
      if ds[ni][i] > maxi: maxi = ds[ni][i]
    unique_vals[i] = maxi+1

  # get number of each value in each att
  # ex: [[2,1,2], [1,3,1], [2,3]] -- 2 red, 1 blue, etc.
  att_cts = []
  for i in range(d): # each att
    cts = [0] * unique_vals[i] 
    for ni in range(n):  # each data item
      v = ds[ni][i]
      cts[v] += 1
    att_cts.append(cts)

  # get number of each value in each att, each cluster
  # ex: k_cts = [ k=0 [[2,0,0], [1,0,1], [1,1]],  
  #               k=1 [[0,1,2], [0,3,0], [1,2]] ]
  k_cts = []
  for k in range(m):  # each cluster
    a_cts = []
    for i in range(d): # each att
      cts = [0] * unique_vals[i] 
      for ni in range(n):  # each data item
        if clustering[ni] != k: continue  # wrong cluster
        v = ds[ni][i]
        cts[v] += 1
      a_cts.append(cts)
    k_cts.append(a_cts) 

  # uncoditional sum squared probs (right summation)
  un_sum_sq = 0.0 
  for i in range(d):  
    for j in range(len(att_cts[i])):
      un_sum_sq += (1.0 * att_cts[i][j] / n) \
      * (1.0 * att_cts[i][j] / n) 

  # conditional sum, each cluster (left summation)
  cond_sum_sq = [0.0] * m  
  for k in range(m):  # each cluster
    sum = 0.0
    for i in range(d):
      for j in range(len(att_cts[i])):
        if cluster_cts[k] == 0: print("FATAL LOGIC ERROR")
        sum += (1.0 * k_cts[k][i][j] / cluster_cts[k]) \
        * (1.0 * k_cts[k][i][j] / cluster_cts[k])
    cond_sum_sq[k] = sum

  # P(C)
  prob_c = [0.0] * m  # [0.0, 0.0]
  for k in range(m):  # each cluster
    prob_c[k] = (1.0 * cluster_cts[k]) / n
  
  # put it all together
  left = 1.0 / m
  right = 0.0
  for k in range(m):
    right += prob_c[k] * (cond_sum_sq[k] - un_sum_sq)
  cu = left * right
  return cu

In [3]:
def cluster(ds, m):
  # ds is encoded
  # greedy algorithm
  n = len(ds)  # number items to cluster

  # assumes first m items are 'different'
  # because they seed the first m clusters
  working_set = [0] * m
  for k in range(m):
    working_set[k] = list(ds[k]) 
    
  clustering = list(range(m))  # [0,1,2, .. m-1]

  for i in range(m, n):
    item_to_cluster = ds[i]
    working_set.append(item_to_cluster)  # working set changed

    proposed_clusterings = []  # empty list
    for k in range(m):         # proposed new clusterings
      copy_of_clustering = list(clustering) 
      copy_of_clustering.append(k)
      proposed_clusterings.append(copy_of_clustering) 

    proposed_cus = [0.0] * m   # compute CU of each proposed
    for k in range(m):
      proposed_cus[k] = \
        cat_utility(working_set, proposed_clusterings[k], m)

    # which proposed clustering will give best CU? (greedy)
    best_proposed = np.argmax(proposed_cus)  # 0, 1, . . m-1

    # update clustering
    clustering.append(best_proposed)

  return clustering

In [11]:
def main():
  print("\nIniciando clusterizacao atraves do metodo C.U. ")
  cursor = connection.cursor()
  connection.commit()

  select_query = """SELECT * from dados_entropicos_user"""
  cursor.execute(select_query)
  teste = cursor.fetchall()
  raw_data = [row for row in teste]
  enc_data = [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[2,0,1,1,1,0,0,1,0,0,0,1,1,1,1,1],[3,0,0,0,2,0,0,0,0,0,0,0,0,0,2,2],[4,1,1,0,3,0,0,0,0,0,0,2,1,1,0,3],[5,1,2,2,4,0,1,1,0,0,0,1,1,1,0,4],[6,0,1,1,5,0,0,1,0,0,0,0,0,1,0,5],[7,1,0,0,6,0,0,0,0,0,0,0,1,0,3,6],[8,0,0,0,7,0,0,0,0,0,0,0,0,0,4,7],[9,1,2,3,8,0,1,1,0,0,0,1,1,1,0,8],[10,1,1,2,9,0,0,0,0,0,0,0,2,2,5,4],[11,1,1,0,10,0,0,0,0,0,0,0,3,3,6,9],[12,1,3,2,11,1,0,2,0,1,0,3,4,4,7,10],[13,0,0,2,12,1,0,2,1,1,0,2,5,5,8,11],[14,1,1,1,13,1,1,3,0,2,1,1,6,6,9,12],[15,1,3,4,14,1,0,2,0,3,0,2,7,5,10,13],[16,1,3,4,15,1,1,2,0,4,0,1,8,4,11,14],[17,0,4,4,16,1,1,0,0,3,0,1,9,7,12,15],[18,0,0,2,17,1,0,4,0,2,1,0,0,8,13,16],[19,0,1,0,18,1,1,3,0,5,1,0,10,9,14,17],[20,1,3,5,19,1,0,5,0,6,0,0,0,7,15,18],[21,1,3,5,20,1,0,5,0,7,0,0,0,10,16,19],[22,1,4,4,21,2,0,5,0,8,0,1,1,11,17,20],[23,0,0,1,22,2,0,2,0,9,0,2,2,10,18,20],[24,1,3,5,23,2,0,2,0,9,0,4,4,11,19,21],[25,1,0,2,24,2,1,3,0,10,1,0,0,12,20,22],[26,1,0,2,24,2,1,3,0,10,1,0,0,12,20,23],[27,1,0,2,24,2,1,3,0,10,1,0,0,12,20,24],[28,1,0,2,24,2,1,3,0,10,1,0,0,12,20,24],[29,1,0,1,25,2,0,1,0,11,0,1,11,13,21,24],[30,1,0,1,26,2,0,6,0,11,1,5,12,14,22,24],[31,1,0,2,27,3,1,7,0,12,0,2,13,0,23,25],[32,0,0,2,28,3,1,7,0,12,0,4,4,4,24,26],[33,0,0,1,29,3,0,0,0,13,0,1,14,0,25,27],[34,1,3,5,30,3,1,4,0,14,1,0,15,15,26,28],[35,0,0,1,29,3,0,0,0,13,0,1,14,0,25,29],[36,0,3,3,31,3,0,6,1,15,1,5,16,16,27,30],[37,0,1,0,32,3,0,3,1,15,1,0,17,8,28,29],[38,1,0,2,24,2,1,3,0,10,1,0,0,12,20,31],[39,1,0,2,24,2,1,3,0,10,1,0,0,12,20,32],[40,1,3,5,33,4,1,0,0,16,0,1,3,4,29,24],[41,1,4,5,34,4,1,2,0,16,0,0,0,17,30,24],[42,1,3,4,35,4,1,0,0,17,0,4,5,18,31,33],[43,0,4,4,36,4,1,0,0,17,0,1,18,5,32,34],[44,0,4,5,37,4,1,2,0,17,0,0,0,17,30,35],[45,1,4,6,38,4,0,3,0,18,1,2,3,19,33,36],[46,0,4,4,39,4,0,2,1,19,0,0,0,17,34,37],[47,1,4,4,40,4,0,2,0,20,0,1,3,17,34,38],[48,1,3,5,41,4,0,3,0,20,1,0,19,20,35,12],[49,0,3,5,42,5,1,5,0,21,0,1,20,18,36,39],[50,0,4,4,43,5,1,8,0,21,1,0,21,21,37,40],[51,0,4,4,44,5,1,5,0,22,0,1,22,5,32,41],[52,1,3,5,45,5,1,2,0,22,0,0,0,4,38,42],[53,1,4,4,46,5,1,2,0,22,0,1,3,5,32,43],[54,1,3,2,47,5,1,2,0,22,0,4,23,22,39,44],[55,1,4,4,48,5,0,9,0,23,1,4,24,23,40,45],[56,1,0,2,49,5,1,2,0,23,0,0,0,4,41,46],[57,1,3,5,50,5,1,0,0,23,0,4,25,4,42,47],[58,0,0,2,51,5,1,2,1,23,0,4,25,24,43,48],[59,1,0,2,52,5,1,0,0,24,0,4,25,24,44,49],[60,0,3,5,53,5,0,3,0,24,1,2,26,23,45,50],[61,1,3,3,54,5,0,10,0,24,1,5,27,25,46,51],[62,1,4,4,55,6,1,2,0,25,0,4,25,24,47,52],[63,1,4,5,56,6,1,2,0,25,0,0,0,26,48,53],[64,1,4,4,57,6,1,2,0,26,0,4,25,24,47,54],[65,1,0,1,58,6,1,0,0,26,0,2,26,17,49,55],[66,0,0,1,59,6,1,0,0,26,0,0,0,17,49,56],[67,0,3,5,60,6,1,0,0,27,0,0,0,4,29,57],[68,1,3,2,61,6,1,0,0,27,0,1,3,27,50,58],[69,1,4,5,62,6,1,2,0,28,0,0,19,28,51,59],[70,1,4,5,63,6,1,2,0,28,0,1,20,28,51,60],[71,1,0,1,64,7,1,5,0,29,0,0,0,29,52,61],[72,0,0,1,65,7,0,3,1,30,1,0,3,23,53,62],[73,1,0,2,66,7,0,6,0,31,1,5,28,30,54,63],[74,1,3,5,67,7,0,7,0,32,0,4,25,31,55,64],[75,1,3,5,68,7,0,8,0,33,1,0,26,19,56,65],[76,1,3,2,69,7,0,8,1,34,1,0,29,32,57,66],[77,1,0,1,70,7,0,1,1,34,0,0,0,33,58,67],[78,0,0,1,71,7,0,1,1,29,0,0,0,33,58,68],[79,1,3,5,72,8,1,2,0,35,0,0,30,4,59,69],[80,1,0,1,73,8,1,2,0,35,0,0,31,34,60,70],[81,0,1,0,74,8,1,5,0,36,0,0,32,26,61,71],[82,1,3,4,75,8,1,5,0,36,0,0,0,35,62,72],[83,1,1,0,76,8,0,5,0,37,0,0,19,36,63,73],[84,1,1,0,77,8,1,5,0,37,0,0,0,37,64,74],[85,0,0,1,78,8,1,5,0,37,0,4,21,33,65,75],[86,1,0,2,24,2,1,3,0,10,1,0,0,12,20,76],[87,0,3,5,79,9,0,3,0,38,1,0,3,19,66,77],[88,1,0,2,80,9,0,7,0,39,0,0,19,27,67,78],[89,1,0,2,81,9,0,0,1,40,0,0,20,26,68,79],[90,0,1,1,82,9,0,0,0,40,0,4,33,38,69,80],[91,1,3,5,83,9,0,0,0,41,0,4,34,4,29,81],[92,1,0,2,84,10,0,7,0,42,0,4,35,27,70,82],[93,1,3,2,85,10,0,2,0,43,0,4,1,38,71,83],[94,0,0,1,86,10,0,11,1,44,1,5,36,39,72,84],[95,1,1,0,87,10,1,1,1,42,0,4,33,27,70,85],[96,0,1,0,88,10,0,3,0,45,1,0,3,32,73,86],[97,0,1,0,88,10,0,3,0,45,1,0,3,32,73,87],[98,1,5,1,83,11,0,3,1,46,1,0,19,40,74,88],[99,0,1,0,89,11,1,2,1,46,0,0,20,28,75,88],[100,1,1,0,89,11,0,0,0,47,0,4,1,28,76,89]]

  print("\nDados sem tratamento: ")
  for item in raw_data :
    print(item)
  print("\nDados normalizados: ")
  for item in enc_data:
    print(item)

  m = 3  # number clusters
  seed_val = 0
  print("\nIniciando clusterizacao com %d clusters" % m)
  clustering = cluster(enc_data, m)
  print("Pronto!")

  print("\nResultado geral: ")
  print(clustering) 

  cu = cat_utility(enc_data, clustering, m)
  print("Category utility of clustering = %0.4f \n" % cu)

  print("\nDados clusterizados sem tratamento: ")
  print("=====")
  for k in range(m):
    for i in range(len(enc_data)):
      if clustering[i] == k:
        print("ID: ",raw_data[i][1], " | Cluster: ", k)
        update_query = """ INSERT INTO kmeans_cluster (cluster, usuario_id) VALUES (%s, %s) """
        cursor.execute(update_query, (k,raw_data[i][1]))
        connection.commit()
    print("=====")

  print("\nFim \n")

if __name__ == "__main__":
  main()


Iniciando clusterizacao atraves do metodo C.U. 

Dados sem tratamento: 
(104, 1, False, 8, 2, '137.183.95.242', 'pt-BR', True, 'Windows 10', True, 'America/Sao_Paulo', False, 'Chrome', '95.0.4638', 'nvidia evga geforce gtx 970', None, '1gd7abb2e497c9f5a00ff94da5dc9e2')
(105, 2, False, 4, 4, '34.131.211.72', 'pt-BR', True, 'Windows 8', True, 'America/Sao_Paulo', False, 'Firefox', '80.0.4170', 'amd radeon rx 560', None, '1adl54148eac5h2035mbf91h81e8i')
(106, 3, False, 8, 2, '201.98.244.196', 'pt-BR', True, 'Windows 10', True, 'America/Sao_Paulo', False, 'Chrome', '95.0.4638', 'nvidia evga geforce gtx 970', None, 'e09c80c42fda55f9d992e59ca6b3307d')
(107, 4, True, 4, 2, '107.5.15.1', 'pt-BR', True, 'Windows 10', True, 'America/Sao_Paulo', False, 'Edge', '80.0.4170', 'amd radeon rx 560', None, '1gd7abb2e497c9f5a00ff94da5dc9e2')
(108, 5, True, 2, 6, '251.203.64.179', 'pt-BR', False, 'Windows 8', True, 'America/Sao_Paulo', False, 'Firefox', '80.0.4170', 'amd radeon rx 560', None, '1gd7abb2e4